In [11]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [12]:
spark = SparkSession.builder \
    .appName('spark-workshop exercises') \
    .getOrCreate()

spark

### Spark SQL

#### Exercise 15: Finding Most Populated Cities Per Country

In [13]:
data = spark.read.csv(
    './data/spark-sql-15-input.csv',
    header=True,
    inferSchema=True
)

data

DataFrame[name: string, country: string, population: string]

In [14]:
data.show()

+-----------------+-------------+----------+
|             name|      country|population|
+-----------------+-------------+----------+
|           Warsaw|       Poland| 1 764 615|
|           Cracow|       Poland|   769 498|
|            Paris|       France| 2 206 488|
|Villeneuve-Loubet|       France|    15 020|
|    Pittsburgh PA|United States|   302 407|
|       Chicago IL|United States| 2 716 000|
|     Milwaukee WI|United States|   595 351|
|          Vilnius|    Lithuania|   580 020|
|        Stockholm|       Sweden|   972 647|
|         Goteborg|       Sweden|   580 020|
+-----------------+-------------+----------+



##### Remove the whitespaces in the values of population and convert them to int.

In [15]:
data = data.withColumn(
    'population',
    F.regexp_replace('population', r'\s', '').cast('int')
)
data

DataFrame[name: string, country: string, population: int]

In [16]:
data.show()

+-----------------+-------------+----------+
|             name|      country|population|
+-----------------+-------------+----------+
|           Warsaw|       Poland|   1764615|
|           Cracow|       Poland|    769498|
|            Paris|       France|   2206488|
|Villeneuve-Loubet|       France|     15020|
|    Pittsburgh PA|United States|    302407|
|       Chicago IL|United States|   2716000|
|     Milwaukee WI|United States|    595351|
|          Vilnius|    Lithuania|    580020|
|        Stockholm|       Sweden|    972647|
|         Goteborg|       Sweden|    580020|
+-----------------+-------------+----------+



In [25]:
max_population_by_country = data.groupBy('country') \
    .agg(F.expr('MAX(population) AS max_population'))

max_population_by_country.show()

+-------------+--------------+
|      country|max_population|
+-------------+--------------+
|       Sweden|        972647|
|       France|       2206488|
|United States|       2716000|
|    Lithuania|        580020|
|       Poland|       1764615|
+-------------+--------------+



In [27]:
data.join(
    max_population_by_country,
    on=(data['country'] == max_population_by_country['country'])
    & (data['population'] == max_population_by_country['max_population'])
).drop('max_population') \
    .drop(max_population_by_country['country']) \
    .show(truncate=False)

+----------+----------+-------------+
|name      |population|country      |
+----------+----------+-------------+
|Warsaw    |1764615   |Poland       |
|Paris     |2206488   |France       |
|Chicago IL|2716000   |United States|
|Vilnius   |580020    |Lithuania    |
|Stockholm |972647    |Sweden       |
+----------+----------+-------------+



##### Spark SQL version

In [33]:
data.createOrReplaceTempView('exercise_15_data')

spark.sql(
    """
    WITH max_population_by_country AS (
        SELECT country, MAX(population) AS max_population
        FROM exercise_15_data
        GROUP BY country
    )
    SELECT
        exercise_15_data.country,
        exercise_15_data.name,
        max_population_by_country.max_population
    FROM exercise_15_data
    INNER JOIN max_population_by_country
        ON exercise_15_data.population = max_population_by_country.max_population
            AND exercise_15_data.country = max_population_by_country.country
    ORDER BY max_population DESC
    """
).show(truncate=False)

+-------------+----------+--------------+
|country      |name      |max_population|
+-------------+----------+--------------+
|United States|Chicago IL|2716000       |
|France       |Paris     |2206488       |
|Poland       |Warsaw    |1764615       |
|Sweden       |Stockholm |972647        |
|Lithuania    |Vilnius   |580020        |
+-------------+----------+--------------+



#### Exercise 25: Collect values per group

In [49]:
exercise_25_data = spark.range(20).withColumn('group', F.col('id') % 2)

exercise_25_data.show()

+---+-----+
| id|group|
+---+-----+
|  0|    0|
|  1|    1|
|  2|    0|
|  3|    1|
|  4|    0|
|  5|    1|
|  6|    0|
|  7|    1|
|  8|    0|
|  9|    1|
| 10|    0|
| 11|    1|
| 12|    0|
| 13|    1|
| 14|    0|
| 15|    1|
| 16|    0|
| 17|    1|
| 18|    0|
| 19|    1|
+---+-----+



In [50]:
exercise_25_data.groupBy('group') \
    .agg(F.collect_list('id')) \
    .show(truncate=False)

+-----+-----------------------------------+
|group|collect_list(id)                   |
+-----+-----------------------------------+
|0    |[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]|
|1    |[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]|
+-----+-----------------------------------+



#### Exercise 33: Calculating Gap Between Current And Highest Salaries Per Department

In [52]:
exercise_33_data = spark.read.csv(
    './data/spark-sql-33-input.csv',
    header=True,
    inferSchema=True
)

exercise_33_data

DataFrame[id: int, name: string, department: string, salary: int]

In [53]:
exercise_33_data.show(truncate=False)

+---+-----------------+----------+------+
|id |name             |department|salary|
+---+-----------------+----------+------+
|1  |Hunter Fields    |IT        |15    |
|2  |Leonard Lewis    |Support   |81    |
|3  |Jason Dawson     |Support   |90    |
|4  |Andre Grant      |Support   |25    |
|5  |Earl Walton      |IT        |40    |
|6  |Alan Hanson      |IT        |24    |
|7  |Clyde Matthews   |Support   |31    |
|8  |Josephine Leonard|Support   |1     |
|9  |Owen Boone       |HR        |27    |
|10 |Max McBride      |IT        |75    |
+---+-----------------+----------+------+



##### PySpark version

In [93]:
exercise_33_data.groupBy('department') \
    .agg(F.max('salary').alias('max_salary')) \
    .join(
        exercise_33_data,
        on='department',
        how='inner'
    ).selectExpr(
        'id',
        'name',
        'department',
        'salary',
        'max_salary - salary AS diff'
    ).show()

+---+-----------------+----------+------+----+
| id|             name|department|salary|diff|
+---+-----------------+----------+------+----+
|  1|    Hunter Fields|        IT|    15|  60|
|  2|    Leonard Lewis|   Support|    81|   9|
|  3|     Jason Dawson|   Support|    90|   0|
|  4|      Andre Grant|   Support|    25|  65|
|  5|      Earl Walton|        IT|    40|  35|
|  6|      Alan Hanson|        IT|    24|  51|
|  7|   Clyde Matthews|   Support|    31|  59|
|  8|Josephine Leonard|   Support|     1|  89|
|  9|       Owen Boone|        HR|    27|   0|
| 10|      Max McBride|        IT|    75|   0|
+---+-----------------+----------+------+----+



##### Spark SQL version

In [55]:
exercise_33_data.createOrReplaceTempView('exercise_33_data')

spark.sql(
    """
    WITH max_salary_by_department AS (
        SELECT department, MAX(salary) AS max_salary
        FROM exercise_33_data
        GROUP BY department
    )
    SELECT
        exercise_33_data.id,
        exercise_33_data.name,
        exercise_33_data.department,
        exercise_33_data.salary,
        max_salary_by_department.max_salary - exercise_33_data.salary AS diff
    FROM exercise_33_data
    INNER JOIN max_salary_by_department
        USING (department)
    """
).show(truncate=False)


+---+-----------------+----------+------+----+
|id |name             |department|salary|diff|
+---+-----------------+----------+------+----+
|1  |Hunter Fields    |IT        |15    |60  |
|2  |Leonard Lewis    |Support   |81    |9   |
|3  |Jason Dawson     |Support   |90    |0   |
|4  |Andre Grant      |Support   |25    |65  |
|5  |Earl Walton      |IT        |40    |35  |
|6  |Alan Hanson      |IT        |24    |51  |
|7  |Clyde Matthews   |Support   |31    |59  |
|8  |Josephine Leonard|Support   |1     |89  |
|9  |Owen Boone       |HR        |27    |0   |
|10 |Max McBride      |IT        |75    |0   |
+---+-----------------+----------+------+----+



##### Spark SQL version alternative

In [58]:
# So it looks like that Spark SQL does not support this type of correlated subquery :/
# Or am I doing something wrong? (TODO find out it)
spark.sql(
    """
    SELECT
        exercise_33_data.id,
        exercise_33_data.name,
        exercise_33_data.department,
        exercise_33_data.salary,
        (SELECT MAX(exercise_33_data_inner.salary) - exercise_33_data.salary
         FROM exercise_33_data AS exercise_33_data_inner
         WHERE exercise_33_data.department = exercise_33_data_inner.department)
    FROM exercise_33_data
    """
).show(truncate=False)

AnalysisException: Expressions referencing the outer query are not supported outside of WHERE/HAVING clauses:
Aggregate [(max(salary#1040) - outer(salary#904)) AS (max(salary) - outer(exercise_33_data.salary))#1036]
+- Filter (outer(department#903) = department#1039)
   +- SubqueryAlias exercise_33_data_inner
      +- SubqueryAlias exercise_33_data
         +- View (`exercise_33_data`, [id#1037,name#1038,department#1039,salary#1040])
            +- Relation [id#1037,name#1038,department#1039,salary#1040] csv
;
Project [id#901, name#902, department#903, salary#904, scalar-subquery#1034 [salary#904 && department#903] AS scalarsubquery(salary, department)#1041]
:  +- Aggregate [(max(salary#1040) - outer(salary#904)) AS (max(salary) - outer(exercise_33_data.salary))#1036]
:     +- Filter (outer(department#903) = department#1039)
:        +- SubqueryAlias exercise_33_data_inner
:           +- SubqueryAlias exercise_33_data
:              +- View (`exercise_33_data`, [id#1037,name#1038,depar